In [1]:
import os
import json
import google.generativeai as genai
from tavily import TavilyClient
from dotenv import load_dotenv
from IPython.display import display, Markdown, JSON
import google.generativeai as genai
import urllib.parse
import json
import os
import requests
import base64
from dotenv import load_dotenv
import os
import sys
from dotenv import load_dotenv



/home/lethal365/northern_lights/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Load environment variables
load_dotenv()

# --- CONFIGURATION ---
# Replace these with your actual keys or ensure they are in your .env file
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") 
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

if not GEMINI_API_KEY or not TAVILY_API_KEY:
    print("⚠️ WARNING: Please set GEMINI_API_KEY and TAVILY_API_KEY in your environment or this cell.")

# Configure Clients
genai.configure(api_key=GEMINI_API_KEY)
#TODO change model
model = genai.GenerativeModel('gemini-2.5-pro') 
tavily = TavilyClient(api_key=TAVILY_API_KEY)

In [16]:
### --- BOLAGSVERKET AUTHENTICATION & API CAllS ---

# Load environment variables from a .env file
load_dotenv()

CLIENT_ID = os.getenv("BOLAGSVERKET_CLIENT_ID", "YOUR_CLIENT_ID")
CLIENT_SECRET = os.getenv("BOLAGSVERKET_CLIENT_SECRET", "YOUR_CLIENT_SECRET")

# Endpoints documented by Bolagsverket
TOKEN_URL = "https://portal.api.bolagsverket.se/oauth2/token"
# Base URL found in documentation for Värdefulla datamängder
API_BASE_URL = "https://gw.api.bolagsverket.se/vardefulla-datamangder/v1"

def get_access_token():
    """Authenticates with Bolagsverket and returns an access token."""
    
    # Encode client_id:client_secret in base64 for Basic Auth header
    creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
    creds_b64 = base64.b64encode(creds.encode("utf-8")).decode("utf-8")

    headers = {
        "Authorization": f"Basic {creds_b64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {
        "grant_type": "client_credentials",
        # needed for read / ping access 
        "scope": "vardefulla-datamangder:read vardefulla-datamangder:ping" 
    }

    try:
        response = requests.post(TOKEN_URL, headers=headers, data=data)
        response.raise_for_status()
        token_data = response.json()
        return token_data["access_token"]
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching token: {e}")
        if response.content:
            print(f"Details: {response.content}")
        return None


def search_company(org_number, token):
    """
    Fetches company information using the provided organization number and access token.
    Args:
        org_number (str or int): The 10-digit tax number of the organization. If the input 
            is not 10 digits, it will be zero-padded to ensure the correct format.
        token (str): The access token required for authentication.
    Returns:
        dict: A dictionary containing the company information retrieved from the API, 
            if the request is successful.
        None: If the request fails, returns None and logs the error details.
    Notes:
        - The organization number should be provided without any dashes (e.g., "5560160680").
        - Ensure that the `API_BASE_URL` variable is defined and points to the correct API endpoint.
        - The function prints detailed error messages to help debug issues with the API response.
    """
    
    url = f"{API_BASE_URL}/organisationer"
    
    # ---------------------------------------------------------
    # Use tax id of comapny -> without - 
    # ---------------------------------------------------------
    payload = {
        "identitetsbeteckning": org_number
    }

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching data: {e}")
        # Print detailed error to debug schema issues
        print(f"Response: {response.text}")
        return None
    


In [20]:
def generate_queries(company_name: str, org_id: str = None) -> list[str]:
    """
    Ask Gemini to generate optimized search queries based on the entity name.
    """
    input_data = f"""
    <user>
        <input>
            <company_info>
                <name>{company_name}</name>
                <company_id>{org_id or "Unknown"}</company_id>
            </company_info>
        </input>
    </user>
    """
    
    with open("../app/services/prompts/query_internet.xml", "r") as file:
        QUERY_INTERNET_PROMPT = file.read()
    full_prompt = QUERY_INTERNET_PROMPT + input_data
    
    response = model.generate_content(
        full_prompt, 
        generation_config={"response_mime_type": "application/json"}
    )
    
    try:
        data = json.loads(response.text)
        # Extract just the query strings from the structured objects
        queries = [q["query"] for q in data.get("queries", [])]
        # Deduplicate
        return list(set(queries))
    except Exception as e:
        print(f"Error parsing Gemini response: {e}")
        return [f"{company_name} official website", f"{company_name} Sweden annual report"]

def perform_search(queries: list[str], max_results: int = 5):
    """
    Step 2: Execute search using Tavily API.
    """
    aggregated_context = []
    
    print(f"🔎 Executing {len(queries)} search queries...")
    
    # To save tokens/API credits, we might limit queries here
    # TODO test values 
    for query in queries[:3]: # Limit to top 3 queries for this demo
        try:
            print(f"   -> Searching: '{query}'")
            # Tavily 'search' returns structured results with content
            response = tavily.search(query=query, search_depth="advanced", max_results=5)
            
            for result in response.get("results", []):
                aggregated_context.append(f"Source: {result['url']}\nContent: {result['content']}\n---")
        except Exception as e:
            print(f"   x Error searching '{query}': {e}")
            
    return "\n".join(aggregated_context)

def structure_data(company_name: str, search_context: str):
    """
    Step 3: Structure the gathered raw text into the Northern Lights JSON schema.
    """
    input_data = f"""
    <user>
        <input>
            <source_data>
                <bolagsverket>
                    Legal Name: {company_name}
                    Registered: Sweden
                </bolagsverket>
                <web_search>
                    {search_context}
                </web_search>
            </source_data>
            
            <entity_context>
                <entity_name>{company_name}</entity_name>
                <entity_type>company</entity_type>
            </entity_context>
        </input>
    </user>
    """
    
    with open("../app/services/prompts/structure_data.xml", "r") as file:
        STRUCTURE_DATA_PROMPT = file.read()
    full_prompt = STRUCTURE_DATA_PROMPT + input_data
    #print(full_prompt)
    response = model.generate_content(
        full_prompt, 
        generation_config={"response_mime_type": "application/json"}
    )
    
    try:
        return json.loads(response.text)
    except Exception as e:
        print(f"Error structuring data: {e}")
        return {"error": response.text}

In [21]:
# --- RUN THE PIPELINE ---

# 1. Define Target
COMPANY_NAME = "Ericsson"
print(f"Starting Scraper for: {COMPANY_NAME}\n")

# TODO add call to api and get org data 
# 2. Generate Queries
queries = generate_queries(COMPANY_NAME)
print("✅ Generated Queries:")
print(json.dumps(queries[:2], indent=2))

# 3. Scrape Web
if queries:
    search_context = perform_search(queries)
    print(f"\n✅ Retrieved {len(search_context)} characters of context.")
else:
    search_context = ""
    print("❌ No queries generated.")


# 4. Structure Data
if search_context:
    print("\n🧠 Structuring data with Gemini...")
    structured_data = structure_data(COMPANY_NAME, search_context)
    
    print("\n✨ FINAL JSON OUTPUT:")
    display(JSON(structured_data))
else:
    print("❌ Skipping structuring due to lack of context.")
    


Starting Scraper for: Ericsson

✅ Generated Queries:
[
  "Ericsson v\u00e4rderingar och syfte",
  "Ericsson industry sector business areas"
]
🔎 Executing 20 search queries...
   -> Searching: 'Ericsson värderingar och syfte'
   -> Searching: 'Ericsson industry sector business areas'
   -> Searching: 'Telefonaktiebolaget LM Ericsson organizationsnummer'

✅ Retrieved 17679 characters of context.

🧠 Structuring data with Gemini...

✨ FINAL JSON OUTPUT:


<IPython.core.display.JSON object>

In [25]:
import sys
import os
from typing import Dict, Any

# Get the current directory of the notebook/script
current_dir = os.getcwd()

# Get the project root (assuming data_pipeline is one level deep)
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Add project root to path so 'app' can be found
if project_root not in sys.path:
    sys.path.append(project_root)

# Now you can import safely
from app.db.neo4j_client import get_driver

def ingest_company_full(data: Dict[str, Any]) -> None:
    """
    Ingests a complete company record including metadata and portfolio relationships.
    Matches the JSON structure provided.
    """
    driver = get_driver()
    
    # 1. Cleaning & Preparation
    # Map 'organization_id' from JSON to 'company_id' for the DB
    company_id = data.get("organization_id")
    if not company_id:
        print(f"Skipping {data.get('name')}: No organization_id")
        return

    # Prepare standard properties
    props = {
        "company_id": company_id,
        "name": data.get("name"),
        "country_code": data.get("country_code", "SE"),
        "description": data.get("description", ""),
        "mission": data.get("mission", ""),
        "year_founded": str(data.get("year_founded") or ""), # Ensure string if needed
        "num_employees": data.get("num_employees"),
        "website": data.get("website", ""),
        "sectors": data.get("sectors", []),
        "aliases": data.get("aliases", []),
        "key_people": data.get("key_people", [])
    }

    # 2. Upsert Company Node Query
    company_query = """
    MERGE (c:Company {company_id: $company_id})
    SET c += $props, 
        c.updated_at = datetime()
    RETURN c.name as name
    """

    # 3. Portfolio Relationship Query
    # This processes the portfolio list. If entity_id is null, it creates a placeholder node based on Name.
    # We assume 'portfolio' items imply an OWNERSHIP or SUBSIDIARY relationship.
    portfolio_query = """
    MATCH (source:Company {company_id: $source_id})
    UNWIND $portfolio as item
    
    # 1. Find or Create the target node
    # Ideally we merge on ID, but if ID is null, we merge on Name (careful with duplicates)
    MERGE (target:Company {name: item.name})
    ON CREATE SET 
        target.company_id = coalesce(item.entity_id, randomUUID()), # Assign ID if missing
        target.created_via_ingest = true
    
    # 2. Create the relationship
    MERGE (source)-[r:OWNS]->(target)
    SET r.source = 'llm_ingest'
    """

    with driver.session() as session:
        # A. Upsert the main Company node
        try:
            session.run(company_query, company_id=company_id, props=props)
            print(f"✅ Upserted Company: {props['name']}")
        except Exception as e:
            print(f"❌ Error upserting company node: {e}")
            return

        # B. Upsert Portfolio Relationships
        portfolio_items = data.get("portfolio", [])
        if portfolio_items:
            try:
                # Filter out empty items
                valid_items = [p for p in portfolio_items if p.get("name")]
                if valid_items:
                    session.run(portfolio_query, source_id=company_id, portfolio=valid_items)
                    print(f"   ↳ Linked {len(valid_items)} portfolio items.")
            except Exception as e:
                print(f"❌ Error linking portfolio: {e}")

# --- EXAMPLE USAGE ---
if __name__ == "__main__":
    # The JSON data you provided
    llm_data = structured_data # TODO 

    ingest_company_full(llm_data)

✅ Upserted Company: Telefonaktiebolaget LM Ericsson


In [26]:
org_id = structured_data.get('organization_id')
print(f"Extracted Organization ID: {org_id}")
print("1️⃣  Authenticating...")
# Call the Bolagsverket API to get more informaiton on the company 
token = get_access_token()
if token:
    print("✅ Access Token received!")
    
    # Example: Search for Bolagsverket's own org number (202100-5489)
    # Remove hyphen for the API: 2021005489
    test_org_number = 9697802230
    
    print(f"2️⃣  Searching for company: {test_org_number}...")
    boglagsverket_api_data = search_company(test_org_number, token)
    
    if boglagsverket_api_data:
        print("✅ Data received:")
        print(boglagsverket_api_data)
else:
    print("🛑 Could not proceed without token.")

Extracted Organization ID: 556016-0680
1️⃣  Authenticating...
❌ Error fetching token: 401 Client Error:  for url: https://portal.api.bolagsverket.se/oauth2/token
Details: b'{"error_description":"A valid OAuth client could not be found for client_id: YOUR_CLIENT_ID","error":"invalid_client"}'
🛑 Could not proceed without token.
